$$ ITI \space AI-Pro: \space Intake \space 45 $$
$$ Recommender \space Systems $$
$$ Lab \space no. \space 3 $$

# `01` Import Necessary Libraries

## `i` Default Libraries

In [29]:
import numpy as np
import pandas as pd

## `ii` Additional Libraries
Add imports for additional libraries you used throughout the notebook

In [1]:
from surprise import Dataset, Reader
from surprise import SVD
from surprise.model_selection import train_test_split
from surprise.accuracy import mae

## `iii` Global Variables


In [ ]:
K = 5
EPOCHS = 100
LR = 0.001
BETA = 0.02
SEED = 1234

----------------------------

# `02` Load Data

Load `songsDataset.csv` file into a dataframe

In [30]:
data = pd.read_csv("Data/songsDataset.csv")
data.head()

,userID,songID,rating
0,0,90409,5
1,4,91266,1
2,5,8063,2
3,5,24427,4
4,5,105433,4


--------------------------

# `03` Matrix Factorization using Gradient Descent

Practice for Matrix Factorization Implementation

**Matrix Factorization Mathematical Derivation**

We know that the matrix factorization breaks the rating matrix $R$ into two matrices $U \in \mathbb{R}^{\#users \times K}$ and $M \in \mathbb{R}^{K \times \#items}$ where K represent the latent space dimensionality.

$R$ can then be approximated through the following equation:

$$
\mathbf{R} \approx \mathbf{U} \times \mathbf{M} = \hat{\mathbf{R}}
$$

The error, incorporating the regularization term, is calculated as follows:

$$
e_{ij}^2 = (r_{ij} - \sum_{k=1}^K{u_{ik}m_{kj}})^2 + \frac{\beta}{2} \sum_{k=1}^K{(||U||^2 + ||M||^2)}
$$

In order to be able to use Stochastic Gradient Descent (SGD) to optimize $U$ and $M$, we need to find the partial derivatives of the error function with respect to both $u_{ik}$ and $m_{kj}$. The partial derivatives can be derived as follows:

$$
\frac{\partial}{\partial u_{ik}}e_{ij}^2 = -2(r_{ij} - \hat{r}_{ij})(m_{kj}) + \frac{\beta}{2} \times (2 u_{ik}) = -2 e_{ij} m_{kj} + \beta u_{ik}
$$

$$
\frac{\partial}{\partial m_{ik}}e_{ij}^2 = -2(r_{ij} - \hat{r}_{ij})(u_{ik}) + \frac{\beta}{2} \times (2 m_{kj}) = -2 e_{ij} u_{ik} + \beta m_{kj}
$$

Thus the update rules will be:

$$
u'_{ik} = u_{ik} + \alpha \frac{\partial}{\partial u_{ik}}e_{ij}^2 = u_{ik} - \alpha(-2 e_{ij} m_{kj} + \beta u_{ik} ) = u_{ik} + \alpha(2 e_{ij} m_{kj} - \beta u_{ik} )
$$

$$
m'_{kj} = m_{kj} + \alpha \frac{\partial}{\partial m_{kj}}e_{ij}^2 = m_{kj} - \alpha(-2 e_{ij} u_{ik} + \beta m_{kj} ) = m_{kj} + \alpha(2 e_{ij} u_{ik} - \beta m_{kj} )
$$

## `0` Construct Utility Matrix from the Data

In [31]:
utility_matrix = data.pivot_table(index='userID', columns='songID', values='rating', fill_value=0)
utility_matrix

songID,2263,2726,3785,8063,12709,13859,16548,17029,19299,19670,...,113954,119103,120147,122065,123176,125557,126757,131048,132189,134732
userID,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199976,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0
199980,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
199988,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## `i` Matrices Initialization

Initialize the two random weights matrices $U$ and $M$ (Try $K=3$)

**Note**: Refer to the next cell for the dimensions of $U$ and $M$

**Hine**: You may use a function from `numpy.random` module (see, [Documentation](https://numpy.org/doc/stable/reference/random/index.html))

In [32]:
U = np.random.rand(utility_matrix.shape[0], K)  
M = np.random.rand(K, utility_matrix.shape[1])  

## `ii` Define a Function to Implement Matrix Factorization

**Function Parameters**:
- `R`: Utility Matrix [of shape: (no. of users, no. of items)]
- `U`: User Latent Features Array [of shape: (no. of users, K)]
- `M`: Items Latent Features Array [of shape: (K, no. of items)]
- `epochs`: No. of training epochs
- `lr`: Learning rate (alpha)
- `beta`: Regularization Parameter

**Function Output**:
- `U`: Optimized User Latent Features Array
- `M`: Optimized Items Latent Features Array

**Main Procedures**:
1. Calculate predicted ratings
2. Calculate MSE Error
3. Calculate gradients
4. Update $U$ and $M$


In [33]:
def matrix_factorization(R: np.ndarray, U: np.ndarray, M: np.ndarray, epochs: int, lr: float, beta: float):
    """
    Function Parameters:
    - `R`: Utility Matrix [of shape: `(no. of users, no. of items)`]
    - `U`: User Latent Features Array [of shape: `(no. of users, K)`]
    - `M`: Items Latent Features Array [of shape: `(K, no. of items)`]
    - `epochs`: No. of training epochs
    - `lr`: Learning rate (alpha)
    - `beta`: Regularization Parameter

    Function Output:
    - `U`: Optimized User Latent Features Array
    - `M`: Optimized Items Latent Features Array
    """

    # Confirm that no. of features is consistent between U and M
    assert U.shape[1] == M.shape[0], f'U and M must have consistent K. Found K={U.shape[1]} for U and K={M.shape[0]} for M'

    # Extract No. of Features (K)
    K = U.shape[1]

    # Define the Epochs loop
    for epoch in range(epochs):
        # Loop over every element in R
        for i in range(R.shape[0]):  # Loop over each user
            for j in range(R.shape[1]):  # Loop over each item
                if R[i, j] > 0:  # Only proceed if the current interaction (i, j) is not missing
                    eij = R[i, j] - np.dot(U[i, :], M[:, j])  # Calculate the error in prediction
                    for k in range(K):  # Loop over each latent features dimension
                        # Update Rules for both U and M:
                        U[i, k] += lr * (2 * eij * M[k, j] - beta * U[i, k])
                        M[k, j] += lr * (2 * eij * U[i, k] - beta * M[k, j])

        ## Error Calculation ##
        e_last = e if epoch > 0 else 100000000
        e = 0  # Initialize a variable to accumulate the errors
        for i in range(R.shape[0]):  # Loop over each user
            for j in range(R.shape[1]):  # Loop over each item
                if R[i, j] > 0:  # Only proceed if the current interaction (i, j) is not missing
                    # since we only calculate the error for interactions having a ground truth value

                    first_part = (R[i, j] - np.dot(U[i, :], M[:, j])) ** 2  # calculate the first part of the error
                    second_part = 0  # Initialize a variable to accumulate the second part of the error

                    for k in range(K):  # Loop over each latent features dimension
                        second_part += (beta / 2) * (U[i, k] ** 2 + M[k, j] ** 2)

                    e += first_part + second_part  # accumulate the error to the total error

        print(f'Epoch {epoch+1}/{epochs}: Total Error = {e}')

        if e < 0.001 or e_last - e < 0.001:  # Stop if error is so small or improvement is not significant
            break

    return U, M


### `[Bonus]` Vectorized Error Calculation

Can the error calculation part be vectorized to get rid of for loops?

If you would like a challenge, try to redefine the function in the next cell with a vectorized error calculation.

In [26]:
def matrix_factorization(R: np.ndarray, U: np.ndarray, M: np.ndarray, epochs: int, lr: float, beta: float):
    """
    Function Parameters:
    - `R`: Utility Matrix [of shape: `(no. of users, no. of items)`]
    - `U`: User Latent Features Array [of shape: `(no. of users, K)`]
    - `M`: Items Latent Features Array [of shape: `(K, no. of items)`]
    - `epochs`: No. of training epochs
    - `lr`: Learning rate (alpha)
    - `beta`: Regularization Parameter

    Function Output:
    - `U`: Optimized User Latent Features Array
    - `M`: Optimized Items Latent Features Array
    """

    # Confirm that no. of features is consistent between U and M
    assert U.shape[1] == M.shape[0], f'U and M must have consistent K. Found K={U.shape[1]} for U and K={M.shape[0]} for M'

    # Extract No. of Features (K)
    K = U.shape[1]

    # Define the Epochs loop
    for epoch in range(epochs):
        # Loop over every element in R
        for i in range(R.shape[0]):  # Loop over each user
            for j in range(R.shape[1]):  # Loop over each item
                if R[i, j] > 0:  # Only proceed if the current interaction (i, j) is not missing
                    eij = R[i, j] - np.dot(U[i, :], M[:, j])  # Calculate the error in prediction
                    for k in range(K):  # Loop over each latent features dimension
                        # Update Rules for both U and M:
                        U[i, k] += lr * (2 * eij * M[k, j] - beta * U[i, k])
                        M[k, j] += lr * (2 * eij * U[i, k] - beta * M[k, j])

        ## Error Calculation ##
        e_last = e if epoch > 0 else 100000000
        prediction = U @ M
        mask = R > 0
        error_matrix = (R - prediction) * mask
        squared_error = np.sum(error_matrix**2)
        regularization = (beta / 2) * (np.sum(U**2) + np.sum(M**2))
        e = squared_error + regularization


        print(f'Epoch {epoch+1}/{epochs}: Total Error = {e}')

        if e < 0.001 or e_last - e < 0.001:  # Stop if error is so small or improvement is not significant
            break

    return U, M

U, M = matrix_factorization(utility_matrix.values, U, M, EPOCHS, LR, BETA)

Epoch 1/100: Total Error = 209086.29765459028
Epoch 2/100: Total Error = 199264.5161769293
Epoch 3/100: Total Error = 190604.75851767053
Epoch 4/100: Total Error = 182528.84078908476
Epoch 5/100: Total Error = 174942.22575153867
Epoch 6/100: Total Error = 167786.95716657213
Epoch 7/100: Total Error = 161022.9009092591
Epoch 8/100: Total Error = 154619.66840180123
Epoch 9/100: Total Error = 148552.48345003583
Epoch 10/100: Total Error = 142800.01278430625
Epoch 11/100: Total Error = 137343.2080888402
Epoch 12/100: Total Error = 132164.67497877934
Epoch 13/100: Total Error = 127248.31862786092
Epoch 14/100: Total Error = 122579.13552168166
Epoch 15/100: Total Error = 118143.08279780106
Epoch 16/100: Total Error = 113926.98902755475
Epoch 17/100: Total Error = 109918.48735515364
Epoch 18/100: Total Error = 106105.96094476072
Epoch 19/100: Total Error = 102478.49548414396
Epoch 20/100: Total Error = 99025.8360387733
Epoch 21/100: Total Error = 95738.34689223516
Epoch 22/100: Total Error = 

## `iii` Use the Function to to Optimize the $U$ and $V$

In [34]:
U, M = matrix_factorization(R=utility_matrix.values, U=U, M=M, epochs=EPOCHS, lr=LR, beta=BETA)

Epoch 1/100: Total Error = 214324.89794190146
Epoch 2/100: Total Error = 204677.68823852786
Epoch 3/100: Total Error = 196080.62432608692
Epoch 4/100: Total Error = 188075.57753761247
Epoch 5/100: Total Error = 180569.2443349804
Epoch 6/100: Total Error = 173500.95182463003
Epoch 7/100: Total Error = 166828.1609254304
Epoch 8/100: Total Error = 160518.61874176317
Epoch 9/100: Total Error = 154546.15995484972
Epoch 10/100: Total Error = 148888.4334926426
Epoch 11/100: Total Error = 143525.65131613982
Epoch 12/100: Total Error = 138439.88473751934
Epoch 13/100: Total Error = 133614.6564180558
Epoch 14/100: Total Error = 129034.69328604167
Epoch 15/100: Total Error = 124685.76778543297
Epoch 16/100: Total Error = 120554.58815563745
Epoch 17/100: Total Error = 116628.71639372416
Epoch 18/100: Total Error = 112896.5022827718
Epoch 19/100: Total Error = 109347.02716600803
Epoch 20/100: Total Error = 105970.05403148964
Epoch 21/100: Total Error = 102755.98204148798
Epoch 22/100: Total Error =

## `iv` Recommend top-K Songs

Recommend top-K ($K=5$) songs for user ($userID=199988$)

Note: Make sure to filter songs they already rated before

In [35]:
user_id = 199988

# Get the index of the user in the utility matrix
user_index = utility_matrix.index.get_loc(user_id)

# Calculate predicted ratings for the user
predicted_ratings = np.dot(U[user_index, :], M)

# Get the songs the user has already rated
rated_songs = utility_matrix.loc[user_id][utility_matrix.loc[user_id] > 0].index

# Filter out songs the user has already rated
recommendations = {song_id: rating for song_id, rating in zip(utility_matrix.columns, predicted_ratings) if song_id not in rated_songs}

# Sort the recommendations by predicted rating in descending order
sorted_recommendations = sorted(recommendations.items(), key=lambda x: x[1], reverse=True)

# Get the top-K song IDs
top_k_songs = [song_id for song_id, _ in sorted_recommendations[:K]]

print(f"Top-{K} Recommended Songs for User {user_id}: {top_k_songs}")

Top-5 Recommended Songs for User 199988: [71582, 122065, 2263, 52611, 13859]


In [36]:
user_id_to_recommend = 199988
num_recommendations = 5

user_index = utility_matrix.index.get_loc(user_id_to_recommend)

# Predict ratings for the user for all songs
predicted_ratings = np.dot(U[user_index, :], M)

# Get the songs the user has already rated
rated_songs = data[data['userID'] == user_id_to_recommend]['songID'].tolist()

predicted_ratings_series = pd.Series(predicted_ratings, index=utility_matrix.columns)

# Filter out songs the user has already rated
unrated_songs_predicted_ratings = predicted_ratings_series.drop(rated_songs, errors='ignore')

# Get the top N songs based on predicted ratings
top_k_recommendations = unrated_songs_predicted_ratings.sort_values(ascending=False).head(num_recommendations)

print(f"Top {num_recommendations} recommended songs for User {user_id_to_recommend}:")
for i, (song_id, predicted_rating) in enumerate(top_k_recommendations.items()):
    print(f"\t -Top {i+1} Song: {song_id}, Predicted Rating: {predicted_rating:.4f}")

Top 5 recommended songs for User 199988:
	 -Top 1 Song: 71582, Predicted Rating: 5.0467
	 -Top 2 Song: 122065, Predicted Rating: 4.9731
	 -Top 3 Song: 2263, Predicted Rating: 4.9458
	 -Top 4 Song: 52611, Predicted Rating: 4.9434
	 -Top 5 Song: 13859, Predicted Rating: 4.9138


------------------------

# `04` Matrix Factorization using SVD Algorithm

Practice for using `SVD` algorithm implementation from `scikit surprise` library.

## `i` Prepare the Data

- Load the Data into `surprise` Dataset
- Split data into train and test


In [42]:
data_surprise = pd.read_csv("Data/songsDataset.csv")

reader = Reader(rating_scale=(0, 15))
data_surprise = Dataset.load_from_df(data_surprise[['userID', 'songID', 'rating']], reader)

trainset, testset = train_test_split(data_surprise, test_size=0.3, random_state=SEED)

## `ii` Model Initialization

Instantiate two models:
- Model with baselines (biases)
- Model without baselines

**Note**: Use `surprise.prediction_algorithms.matrix_factorization.SVD` (see, [Documentation](https://surprise.readthedocs.io/en/stable/matrix_factorization.html#:~:text=surprise.prediction_algorithms.matrix_factorization.-,SVD,-(n_factors%3D)))

In [43]:
# Biased Model
baised_model = SVD(n_factors=100, n_epochs=20, biased=True, init_mean=0, init_std_dev=0.1, lr_all=0.005, reg_all=0.02)

In [44]:
# Non-Biased Model
non_baised_model = SVD(n_factors=100, n_epochs=20, biased=False, init_mean=0, init_std_dev=0.1, lr_all=0.005, reg_all=0.02)

## `iii` Fit each Model on Training Data

In [45]:
# Biased Model
baised_model.fit(trainset)

In [46]:
# Non-Biased Model
non_baised_model.fit(trainset)

## `iv` Test both Models on the Testing Data

Compare the errors of the two models using multiple error formulas.

**Note**: Refer to `surprise.accuracy` module (see, [Documentation](https://surprise.readthedocs.io/en/stable/accuracy.html))

In [51]:
# Biased Model
predicted_rating = baised_model.test(testset)
mae(predicted_rating, verbose=True)

MAE:  1.2969


1.2969336878552074

In [52]:
# Non-Biased Model
predicted_rating = non_baised_model.test(testset)
mae(predicted_rating, verbose=True)

MAE:  1.9338


1.933818206642214

## `v` Recommend Top $10$ Songs for User $199988$

Is there a difference in recommended songs from the two models?

In [53]:
# Biased Model

# Get all unique song IDs from the testset
all_song_ids = set([song_rating[1] for song_rating in testset])
# Get the song IDs that the specific user has rated in the testset
rated_song_ids = set([song_rating[1] for song_rating in testset if song_rating[0] == user_id])
# Find the song IDs that the user has not rated in the testset
unrated_song_ids = list(all_song_ids - rated_song_ids)

predictions = [baised_model.predict(user_id, song_id) for song_id in unrated_song_ids]

predictions_df = pd.DataFrame([(pred.iid, pred.est) for pred in predictions], columns=['songID', 'predicted_rating'])
top_10 = predictions_df.sort_values(by='predicted_rating', ascending=False).head(10)

print(f"Top 10 recommended songs for User {user_id} (Biased Model):")
top_10.head(10).style.background_gradient(cmap='Oranges')


Top 10 recommended songs for User 199988 (Biased Model):


,songID,predicted_rating
8,112023,4.947587
39,105433,4.801477
43,132189,4.584482
16,126757,4.427629
53,8063,4.398302
28,3785,4.369807
35,56660,4.312062
20,60465,4.300019
32,36561,4.178467
6,94604,4.172563


In [54]:
# Non-Biased Model

# Get all unique song IDs from the testset
all_song_ids = set([song_rating[1] for song_rating in testset])
# Get the song IDs that the specific user has rated in the testset
rated_song_ids = set([song_rating[1] for song_rating in testset if song_rating[0] == user_id])
# Find the song IDs that the user has not rated in the testset
unrated_song_ids = list(all_song_ids - rated_song_ids)

predictions = [non_baised_model.predict(user_id, song_id) for song_id in unrated_song_ids]

predictions_df = pd.DataFrame([(pred.iid, pred.est) for pred in predictions], columns=['songID', 'predicted_rating'])
top_10 = predictions_df.sort_values(by='predicted_rating', ascending=False).head(10)

print(f"Top 10 recommended songs for User {user_id} (Non Biased Model):")
top_10.head(10).style.background_gradient(cmap='Oranges')


Top 10 recommended songs for User 199988 (Non Biased Model):


,songID,predicted_rating
35,56660,4.694402
2,43267,2.875574
16,126757,2.739632
42,68572,2.033218
26,94535,1.945414
31,92881,1.811115
28,3785,1.621878
44,25182,1.531674
1,52611,1.180511
52,125557,1.110594


----------------------------------------------

$$ Wish \space you \space all \space the \space best \space ♡ $$
$$ Abdelrahman \space Eid $$